In [1]:
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib
import os
import pymorphy2
import re
from copy import deepcopy
import operator
import numpy as np

RusLem = pymorphy2.MorphAnalyzer()

def bprint(l, sep = " "):
    print sep.join(l)

In [26]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)
 

#get list of sentences from html pages
def prepare_html_pages(html_dir):
    html_files = os.listdir(html_dir)
    sentences = []
    for filename in html_files:
        text = "" 
        with open(html_dir + filename, "r") as f:
            for line in f:
                text += line
                
        visible_text = text_from_html(text)
        sentences += visible_text.strip().split(".")
    
    return sentences


#sentences_normaliation
def normalization(sentences):
    for i, sent in enumerate(sentences):
        tokens = re.findall('[\w]+', sent.strip().lower(), re.U)
        sentences[i] = " ".join([RusLem.parse(token)[0].normal_form for token in tokens])
    return sentences


#creating forward index {sent_id: list of word} and token_dictionary
def create_forward_index(sentences):
    forward_index = {}
    token_list = set()
    for sent_id, sent in enumerate(sentences):
        if 1 < len(sent.strip().split()) <= 40:
            forward_index[sent_id] = sent.strip().split()
            for token in sent.strip().split():
                token_list.add(token)
        
    return forward_index, list(token_list)


def calculate_idf(forward_index, token_list):
    cnt_sentences = len(forward_index)
    token_df = {token:0 for token in token_list}
    for tokens in forward_index.values():
        for token in tokens:
            token_df[token] += 1
    token_idf = {token:np.log(cnt_sentences / float(token_df[token])) for token in token_df}
    return token_idf

def vec_normalization(vector):
    return vector / (1.0 * np.sqrt(np.sum(vector**2)))

def cos_similarity(vec_1, vec_2):
    return np.sum(vec_1 * vec_2) / float(np.sqrt(np.sum(vec_1**2)) * np.sqrt(np.sum(vec_2**2)))
    
class Collection(object):
    def __init__(self, htmls_dir="./htmls/"):
        self.raw_sentences = prepare_html_pages(htmls_dir)
        print len(self.raw_sentences)
        self.norm_sentences = normalization(deepcopy(self.raw_sentences))
        print len(self.norm_sentences)
        self.forward_index, self.token_list = create_forward_index(self.norm_sentences)
        print len(self.forward_index)
        self.token_idf = calculate_idf(self.forward_index, self.token_list)
        
    def doc2vec(self, sentence, idf=False):
        """
        Create sentence normalized vector
        """
        vector = [sentence.count(token) for token in self.token_list]
#         print sum(vector)
        if idf:
            vector = [c * np.log(self.token_idf[self.token_list[i]])
                      for i, c in enumerate(vector)]
            
        return np.asarray(vector)
        
    def sents2vectors(self):
        self.simple_vectors = {sent_idx:vec_normalization(
                                        self.doc2vec(self.forward_index[sent_idx], idf=False)
                                        )
                               for sent_idx in self.forward_index.keys()}
        
        self.idf_vectors = {sent_idx:vec_normalization(
                                    self.doc2vec(self.forward_index[sent_idx], idf=True)
                                    )
                               for sent_idx in self.forward_index.keys()}
        
    def ranking(self, query):
        norm_query = normalization([query])[0]
        bprint([norm_query])
        query_vec_simple = self.doc2vec(norm_query.strip().split(), idf=False)
        query_vec_idf = self.doc2vec(norm_query.strip().split(), idf=True)

        rank_simple = {}
        rank_tfidf = {}
        for sent_idx in self.simple_vectors.keys():
            rank_simple[sent_idx] = cos_similarity(query_vec_simple,
                                                   self.simple_vectors[sent_idx])
            rank_tfidf[sent_idx] = cos_similarity(query_vec_idf,
                                                   self.idf_vectors[sent_idx])
        
        return rank_simple, rank_tfidf
            

def print_serp(rank, query, sent_cnt = 10):
    sorted_rank = sorted(rank.items(), key = operator.itemgetter(1), reverse=True)
    print "Query:",
    bprint([query])
    print "\n"
    serp = []
    
    for idx, pair in enumerate(sorted_rank):
#         print idx, pair
        if idx > sent_cnt:
            break
        print pair[1], "\t", pair[0], 
        bprint(collection.forward_index[pair[0]])
        print
        serp.append(pair[0])
    return serp

In [13]:
collection = Collection()
collection.sents2vectors()

1169
1169
933


In [14]:
query_1 = u"В начале XX века немецкий учитель математики, сам того не желая, научил лошадь считать."
query_2 = u"Искусственный язык для фантастической вселенной «Звёздный путь» создал профессиональный лингвист"
query_3 = u"На территории России были обнаружены останки ниппонозаврa, амурозаврa,целурозаврa и другие виды динозавров."

In [15]:
rank_simple_1, rank_idf_1 = collection.ranking(query=query_1)
rank_simple_2, rank_idf_2 = collection.ranking(query=query_2)
rank_simple_3, rank_idf_3 = collection.ranking(query=query_3)

в начало xx век немецкий учитель математика сам тот не желать научить лошадь считать
искусственный язык для фантастический вселенная звёздный путь создать профессиональный лингвист
на территория россия быть обнаружить останки ниппонозаврa амурозаврa целурозаврa и другой вид динозавр


In [27]:
serp_simple_1 = print_serp(rank_simple_1, query_1)

Query: В начале XX века немецкий учитель математики, сам того не желая, научил лошадь считать.


0.284747398726 	111 в начало век популярный в сша стать блюз и джаз который сохранять свой господство в музыка до появление рок н ролл в 1950 х год

0.283069258536 	71 содержание скрыть 1 основной событие 2 главный изобретение 3 использование сочетание xx век в название 4 двадцатый век в искусство 5 смотреть

0.283069258536 	205 также править править вика текст в викитека есть текст по тема документ xx век xx век хронология изобретение

0.283069258536 	1102 xxii век править править вика текст в начало xxii век в клингонский общество повсеместно усилиться влияние класс воин

0.277350098113 	824 в многий страна в тот число и в россия в 21 век наблюдаться тенденция к снижение престижность педагогический профессия и как следствие недооценённость учительский труд

0.273861278753 	197 в сша один из крупный киностудия называться xx век фокс

0.258198889747 	838 учитель в образовательный процесс пр

In [28]:
serp_idf_1 = print_serp(rank_idf_1, query_1)

Query: В начале XX века немецкий учитель математики, сам того не желая, научил лошадь считать.


0.30095035986 	549 ганс считать

0.242162149102 	561 другой слово ганс быть действительно феноменально умный лошадь и прекрасно понимать что от он хотеть но конечно ни математика ни немецкий язык он не знать и не понимать

0.217894923247 	205 также править править вика текст в викитека есть текст по тема документ xx век xx век хронология изобретение

0.185836111712 	544 комиссия возглавить философ и психолог карл штумпф в состав её войти самый разный человек чей профессия быть так или иначе связать с лошадь математика или психология врач ветеринар владелец цирк офицер кавалерия несколько школьный учитель математика и директор берлинский зоопарк

0.16384848594 	198 название популярный советский боевик пират xx век

0.151959371158 	189 начало эпидемия спид

0.149983079329 	197 в сша один из крупный киностудия называться xx век фокс

0.147979632671 	607 некоторый считать что дромеозаврид групп

In [30]:
serp_simple_2 = print_serp(rank_simple_2, query_2)

Query: Искусственный язык для фантастической вселенной «Звёздный путь» создал профессиональный лингвист


0.36514837167 	234 звёздный путь англ

0.363912671437 	371 он известный как сценарист другой сериал в вселенная звёздный путь звёздный путь глубокий космос 9 1993 1999 звёздный путь в яджер 1995 2001

0.350823207723 	255 вселенная звёздный путь один из наиболее детально проработать вымышленный вселенная 1

0.34749779421 	374 он являться режиссёр и сценарист три фильм из серия звёздный путь звёздный путь ii гнев хан 1982 звёздный путь iv путешествие домой 1986 звёздный путь vi неоткрытый страна 1991

0.338061701891 	356 основный статья звёздный путь энтерпрайза звёздный путь энтерпра йз англ

0.335410196625 	1137 он создать специально для фильм лингвист марк окранд

0.316227766017 	4 2 звёздный путь 1

0.310086836473 	1 marc okrand mɑrk ˈoʊkrænd 3 июль 1948 американский лингвист известный как создатель язык для народ фантастический мир кинематограф сша клингонский вулканский и атлан

In [31]:
serp_idf_2 = print_serp(rank_idf_2, query_2)

Query: Искусственный язык для фантастической вселенной «Звёздный путь» создал профессиональный лингвист


0.349282738206 	1137 он создать специально для фильм лингвист марк окранд

0.288200424825 	1 marc okrand mɑrk ˈoʊkrænd 3 июль 1948 американский лингвист известный как создатель язык для народ фантастический мир кинематограф сша клингонский вулканский и атлантский язык

0.243760095689 	255 вселенная звёздный путь один из наиболее детально проработать вымышленный вселенная 1

0.222160411704 	371 он известный как сценарист другой сериал в вселенная звёздный путь звёздный путь глубокий космос 9 1993 1999 звёздный путь в яджер 1995 2001

0.215633043521 	937 можно выделить следующий уровень профессиональный подготовка педагог существующий в настоящее время уровень профессиональный ориентация педагогический класс школа среднее профессиональный образование педагогический колледж высокий профессиональный образование высокий учебный заведение подготовка научно педагогический кадр для высокий

In [32]:
serp_simple_3 = print_serp(rank_simple_3, query_3)

Query: На территории России были обнаружены останки ниппонозаврa, амурозаврa,целурозаврa и другие виды динозавров.


0.362738125055 	467 первый и единственный неполный скелет быть обнаружить японец в 1934 год на территория больница в синегорск сахалин

0.3 	886 уметь слушать и слышать другой и иной мнение

0.283980917124 	141 транспорт тысячелетие основать на конный тяга быть на протяжение хх век заменить на грузовой автомобиль и автобус что стать возможный благодаря крупномасштабный эксплуатация ископаемое топливо

0.282842712475 	839 скульптура учитель на территория мади

0.279751442472 	529 среди вопрос на который он давать ответ быть не только такой как сколько быть 12 12 но и например если восьмой день месяц приходиться на вторник то какой день по счёт быть следующий пятница

0.273861278753 	20 для клингонский важно быть быть непохожий на привычный земной язык

0.258198889747 	521 результат исследование пфунгст быть принять научный сообщество и использоваться в эксперимент по инте

In [33]:
serp_idf_3 = print_serp(rank_idf_3, query_3)

Query: На территории России были обнаружены останки ниппонозаврa, амурозаврa,целурозаврa и другие виды динозавров.


0.286927874807 	467 первый и единственный неполный скелет быть обнаружить японец в 1934 год на территория больница в синегорск сахалин

0.24572800686 	839 скульптура учитель на территория мади

0.21050651436 	659 единственный известный вид amurosaurus riabinini назвать в честь покойный палеонтолог анатолий рябинин который возглавить первый русский экспедиция 1916 и 1917 год для поиск окаменелый останки динозавр 4 5

0.159383066239 	885 помогать другой

0.149710852665 	605 орнитомимозавр немалоизвестный похожий на страус динозавр

0.148047065197 	155 пандемия а в конец век быть обнаружить новое вирусный заболевание спид который возникнуть в африка

0.147064411445 	656 amurosaurus родиться птицетазовый динозавр из подсемейство ламбеозаврин жить в конец меловой период верхний маастрихта 2 найти в россия в благовещенск 3

0.143146067901 	664 быть вскрыть лишь небольшой часть

In [24]:
#Размечаю истинную релевантность предложений для 1-го запроса: {doc_id:relevance}
true_relevance_1 = {111:0, 71:0, 205:0, 1102:0, 824:0, 197:0, 838:0, 859:0, 156:0, 835:0, 115:0,
                     549:1, 561:2, 205:0, 544:1, 198:0, 189:0, 607:0, 899:0, 196:0, 126:0}

In [23]:
#Размечаю истинную релевантность предложений для 2-го запроса: {doc_id:relevance}
true_relevance_2 = {234:1, 371:1, 255:1, 374:1, 356:1, 1137:2, 4:1, 1:2, 434:1, 1068:2, 238:1,
                   937:0, 260:1}  

In [22]:
#Размечаю истинную релевантность предложений для 3-го запроса: {doc_id:relevance}
true_relevance_3 = {467:1, 886:0, 141:0, 839:0, 529:0, 20:0, 521:0, 605:1, 892:1, 145:0, 598:1,
                   659:2, 885:0, 155:0, 656:2, 664:2, 771:0, 231:0, 634:1}  

In [25]:
def ndcg(y_true, y_score, N = 10):
#     y_true = y_true.ravel()
#     y_score = y_score.ravel()
    y_true_sorted = sorted(y_true, reverse=True)
    ideal_dcg = 0
    for i in range(N):
        ideal_dcg += (2 ** y_true_sorted[i] - 1.) / np.log2(i + 2)
        
    dcg = 0
    argsort_indices = np.argsort(y_score)[::-1]
    for i in range(N):
        dcg += (2 ** y_true[argsort_indices[i]] - 1.) / np.log2(i + 2)
    if ideal_dcg == 0:
        return 1
    ndcg = dcg / ideal_dcg
    return ndcg

In [52]:
def ideal_dcg(marks, N=10):
    marks_sorted = sorted(marks, reverse=True)
    ideal_dcg = 0
    for i in range(N):
        ideal_dcg += (1.0 * marks_sorted[i]) / np.log2(i + 2)
    return ideal_dcg
        
def ndcg(marks, ideal_dcg, N=10):
    dcg = 0
    for i in range(N):
        dcg += (1.0 * marks[i]) / np.log2(i + 2)
    if ideal_dcg == 0:
        return 1
    
    ndcg = dcg / float(ideal_dcg)
    return ndcg

In [53]:
rank_true_simple_1 = [true_relevance_1[doc_id] for doc_id in serp_simple_1]
rank_true_idf_1 = [true_relevance_1[doc_id] for doc_id in serp_idf_1]
rank_true_simple_2 = [true_relevance_2[doc_id] for doc_id in serp_simple_2]
rank_true_idf_2 = [true_relevance_2[doc_id] for doc_id in serp_idf_2]
rank_true_simple_3 = [true_relevance_3[doc_id] for doc_id in serp_simple_3]
rank_true_idf_3 = [true_relevance_3[doc_id] for doc_id in serp_idf_3]

rank_true_1 = [true_relevance_1[doc_id] for doc_id in true_relevance_1.keys()]
rank_true_2 = [true_relevance_2[doc_id] for doc_id in true_relevance_2.keys()]
rank_true_3 = [true_relevance_3[doc_id] for doc_id in true_relevance_3.keys()]

In [54]:
ideal_dcg_1 = ideal_dcg(rank_true_1)
ideal_dcg_2 = ideal_dcg(rank_true_2)
ideal_dcg_3 = ideal_dcg(rank_true_3)

print "Ideal NDCG for query 1:", ideal_dcg_1
print "Ideal NDCG for query 2:", ideal_dcg_2
print "Ideal NDCG for query 3:", ideal_dcg_3

Ideal NDCG for query 1: 3.13092975357
Ideal NDCG for query 2: 6.67448909166
Ideal NDCG for query 3: 6.08439426968


In [44]:
print len(rank_true_simple_2)
print len(serp_simple_1)

11
11


In [57]:
ndcg_simple_1 = ndcg(rank_true_simple_1, ideal_dcg_1)
ndcg_idf_1 = ndcg(rank_true_idf_1, ideal_dcg_1)
ndcg_simple_2 = ndcg(rank_true_simple_2, ideal_dcg_2)
ndcg_idf_2 = ndcg(rank_true_idf_2, ideal_dcg_2)
ndcg_simple_3 = ndcg(rank_true_simple_3, ideal_dcg_3)
ndcg_idf_3 = ndcg(rank_true_idf_3, ideal_dcg_3)

print "NDCG for simple system, query 1:", ndcg_simple_1
print "NDCG for simple system, query 2:", ndcg_simple_2
print "NDCG for simple system, query 3:", ndcg_simple_3

print "NDCG for system with idf, query 1:", ndcg_idf_1
print "NDCG for system with idf, query 2:", ndcg_idf_2
print "NDCG for system with idf, query 3:", ndcg_idf_3

NDCG for simple system, query 1: 0.0
NDCG for simple system, query 2: 0.824676788397
NDCG for simple system, query 3: 0.265678849989
NDCG for system with idf, query 1: 0.859979711185
NDCG for system with idf, query 2: 0.91706938669
NDCG for system with idf, query 3: 0.605557277219


In [58]:
mean_ndcg_simple = np.mean([ndcg_simple_1, ndcg_simple_2, ndcg_simple_3])
mean_ndcg_idf = np.mean([ndcg_idf_1, ndcg_idf_2, ndcg_idf_3])

In [59]:
print "Mean NDCG for simple system:", mean_ndcg_simple
print "Mean NDCG for system with idf:", mean_ndcg_idf

Mean NDCG for simple system: 0.363451879462
Mean NDCG for system with idf: 0.794202125031
